## **Mount Drive**

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/VIT/Tamil Argumentation

/content/gdrive/MyDrive/VIT/Tamil Argumentation


## **Install**

In [3]:
pip install transformers nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 127.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.7 MB/s eta 0:00:00


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## **Import Libraries**

In [5]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from copy import deepcopy

from sklearn import metrics
from sklearn.model_selection import KFold

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer,AutoModel,BertTokenizer

from nltk.tokenize import word_tokenize

## **Import Dataset**

In [6]:
df = pd.read_excel("/content/gdrive/MyDrive/VIT/Tamil Argumentation/Twitter Comment Dataset.xlsx")

In [7]:
df.head()

,S No,Tweet,Date of Tweet,Topic,Parent Tweet,Language,Quality,Stance,Argument,Comment,Responding to Tone,Discussing Writer Characteristics,Remark,Relevancy
0,1,"Bro imagine today is Friday , big star movie i...",2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",ENGLISH,Med,Undetermined,0,1,0,0,0,Relevant
1,2,Dei unnoda akkarai TN mela not on others and w...,2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",ENGLISH,Med,Against,0,1,0,1,0,Relevant
2,3,En ninga ivara matum mention panuringa naraiya...,2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",CODE-MIXED,Med,For,0,1,0,0,0,Relevant
3,4,What is happening in Thoothukudi is totally no...,2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",ENGLISH,High,Against,1,1,0,0,0,Relevant
4,5,Ungaluku Sterlite protest prachanaya illa Bala...,2018-05-22,Jalikattu - Ban or Allow,"And tamil people, jalikattu maadu for money an...",CODE-MIXED,Med,Undetermined,0,0,1,0,0,Relevant


In [8]:
df.columns

Index(['S No', 'Tweet', 'Date of Tweet', 'Topic', 'Parent Tweet', 'Language',
       'Quality', 'Stance', 'Argument', 'Comment', 'Responding to Tone',
       'Discussing Writer Characteristics', 'Remark', 'Relevancy'],
      dtype='object')

## **Load Text and Labels**

In [29]:
text = df["Tweet"].to_numpy()
pt = df["Parent Tweet"].to_numpy()
topic = df["Topic"].to_numpy()

Language_label = df["Language"].to_numpy()
Stance_label = df["Stance"].to_numpy()
Quality_label = df["Quality"].to_numpy()
Argument_label = df["Argument"].to_numpy()
Comment_label = df["Comment"].to_numpy()
Writer_label = df["Discussing Writer Characteristics"].to_numpy()
Tone_label = df["Responding to Tone"].to_numpy()
Remark_label = df["Remark"].to_numpy()
Relevancy_label = df["Relevancy"].to_numpy()

## **Label Encoding**

In [28]:
encode_dict_quality = {
    "High": np.array([1, 0, 0]),
    "Med": np.array([0, 1, 0]),
    "Low": np.array([0, 0, 1]),
}

encode_dict_language = {
    "ENGLISH": np.array([1, 0, 0]),
    "TAMIL": np.array([0, 1, 0]),
    "CODE-MIXED": np.array([0, 0, 1]),
}

encode_dict_stance = {
    "For": np.array([1, 0, 0]),
    "Against": np.array([0, 1, 0]),
    "Undetermined": np.array([0, 0, 1]),
}

encode_dict = {
    0: np.array([1, 0]),
    1: np.array([0, 1]),
}

encode_dict_relevancy = {
    "Relevant": np.array([1, 0]),
    "Irrelevant": np.array([0, 1]),
}

In [30]:
Language_label = np.array([encode_dict_language[label] for label in Language_label])
Quality_label = np.array([encode_dict_quality[label] for label in Quality_label])
Argument_label = np.array([encode_dict[label] for label in Argument_label])
Comment_label = np.array([encode_dict[label] for label in Comment_label])
Writer_label = np.array([encode_dict[label] for label in Writer_label])
Tone_label = np.array([encode_dict[label] for label in Tone_label])
Remark_label = np.array([encode_dict[label] for label in Remark_label])
Relevancy_label = np.array([encode_dict_relevancy[label] for label in Relevancy_label])
Stance_label = np.array([encode_dict_stance[label] for label in Stance_label])

## **Pre-Config for mBERT**

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

MAX_LEN = np.max([(len(x)+len(y)+1) for x,y in zip(text,pt)])
MAX_LEN = np.min([MAX_LEN, 510])

BATCH_SIZE = 32
LEARNING_RATE = 1e-1

In [16]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

## **Build Dataset for mBERT**

In [31]:
class ModelDataset(Dataset):
    def __init__(self, X, PT, topic, y, tokenizer, max_len):
        self.max_len = max_len
        self.text = X
        self.topic = topic
        self.pt = PT
        self.tokenizer = tokenizer
        self.targets = y

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        text = self.text[index]
        pt = self.pt[index]
        topic = self.topic[index]

        concat_text = f"{topic}. {pt}"

        inputs = self.tokenizer(
            topic,text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True)

        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        return {
            'ids': torch.tensor(ids, dtype=torch.long).to(device),
            'mask': torch.tensor(mask, dtype=torch.long).to(device),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long).to(device),
            'targets': torch.tensor(self.targets[index], dtype=torch.float).to(device),
        }

## **Build Model**

In [24]:
class CustomModel(nn.Module):

    def __init__(self):
        super(CustomModel, self).__init__()
        self.bert = AutoModel.from_pretrained('bert-base-multilingual-cased')

        for param in self.bert.parameters():
            param.requires_grad = False

        self.out_layer = nn.Linear(768, 2)

    def forward(self, ids, mask, token_type_ids):
        _, features = self.bert(
            ids, token_type_ids=token_type_ids,
            attention_mask=mask, return_dict=False
        )

        output = self.out_layer(features)

        return output

## **Train Model**

In [21]:
def train(epoch, model, train_loader, loss_fn, optimizer):

    model.train()

    for batch in tqdm(train_loader):

        optimizer.zero_grad()

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()


    print(f'Epoch: {epoch + 1}, Loss:  {loss.item()}')

In [22]:
def validation(data_loader, model):

    model.eval()
    targets = []
    outputs = []

    with torch.no_grad():

        for batch in data_loader:

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
            batch_targets = batch['targets'].to(device, dtype = torch.float)

            batch_outputs = model(ids, mask, token_type_ids)

            targets.extend(batch_targets.cpu().numpy().tolist())
            outputs.extend(batch_outputs.cpu().numpy().tolist())

    return outputs, targets

# Model initialization

In [33]:
kf = KFold(n_splits=5)

model_targets = []
model_labels = []

model = CustomModel().to(device)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

In [37]:
ksi = 0

for train_index, test_index in kf.split(text):

    text_train, text_test = text[train_index], text[test_index]
    pt_train, pt_test = pt[train_index], pt[test_index]
    topic_train, topic_test = topic[train_index], topic[test_index]
    labels_train, labels_test = Argument_label[train_index], Argument_label[test_index]

    train_data = ModelDataset(text_train, pt_train, topic_train, labels_train, tokenizer, MAX_LEN)
    test_data = ModelDataset(text_test, pt_test, topic_test, labels_test, tokenizer, MAX_LEN)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

    best_score = -np.inf
    best_weights = None

    EPOCHS = 10

    print(f"Split   ---->    {ksi}")
    ksi+=1

    torch.save(model.state_dict(),"/content/gdrive/MyDrive/VIT/Tamil Argumentation/twitter_model_argument_w3i.pth")

    for epoch in range(EPOCHS):

        train(epoch, model, train_loader, loss_fn, optimizer)
        outputs, targets = validation(test_loader, model)

        print(outputs)
        print(" ")
        print(targets)

        targets = [np.argmax(x) for x in targets]
        outputs = [np.argmax(x) for x in outputs]

        score = metrics.f1_score(targets, outputs, average='weighted')

        if score > best_score:
            best_score = score
            best_weights = deepcopy(model.state_dict())

    model.load_state_dict(best_weights)
    torch.save(model.state_dict(),"/content/gdrive/MyDrive/VIT/Tamil Argumentation/twitter_model_argument_w3i.pth")

    outputs, targets = validation(test_loader, model)

    targets = [np.argmax(x) for x in targets]
    outputs = [np.argmax(x) for x in outputs]

    model_targets.extend(targets)
    model_labels.extend(outputs)

Split   ---->    0


100%|██████████| 34/34 [00:36<00:00,  1.08s/it]


Epoch: 1, Loss:  0.3411031663417816
[[3.1479673385620117, -3.0888469219207764], [2.589803695678711, -2.39440655708313], [2.376863718032837, -2.355923652648926], [1.4896090030670166, -1.455754280090332], [3.5254507064819336, -3.36191987991333], [3.2451093196868896, -3.109168291091919], [3.699712038040161, -3.6374881267547607], [2.9497122764587402, -2.9423482418060303], [3.003610372543335, -2.9090230464935303], [2.3716626167297363, -2.368143081665039], [2.6134448051452637, -2.512491464614868], [2.6911160945892334, -2.58701491355896], [2.7073233127593994, -2.5059850215911865], [3.6065895557403564, -3.682535171508789], [2.5167579650878906, -2.4555859565734863], [1.5779625177383423, -1.48707914352417], [1.577839970588684, -1.495788812637329], [4.383692264556885, -4.618302822113037], [2.889254570007324, -2.7863707542419434], [3.152040481567383, -3.1563198566436768], [1.504806399345398, -1.5263361930847168], [2.013200521469116, -1.9716715812683105], [4.034976959228516, -4.03028678894043], [3.

100%|██████████| 34/34 [00:36<00:00,  1.08s/it]


Epoch: 2, Loss:  0.3869609832763672
[[2.755236864089966, -2.6680045127868652], [2.6076769828796387, -2.4764044284820557], [2.2116589546203613, -2.1755292415618896], [1.1255154609680176, -1.051560640335083], [3.641409158706665, -3.5212297439575195], [3.263543128967285, -3.1494333744049072], [3.6035759449005127, -3.528956174850464], [2.8373684883117676, -2.8137314319610596], [3.184795618057251, -3.103463649749756], [2.1277639865875244, -2.0865654945373535], [2.5496323108673096, -2.466221332550049], [2.4336154460906982, -2.340654134750366], [2.7030351161956787, -2.5623435974121094], [3.499269723892212, -3.459540605545044], [2.3117306232452393, -2.2437613010406494], [1.3774152994155884, -1.3010647296905518], [1.3661775588989258, -1.2954978942871094], [5.121016502380371, -5.052586555480957], [2.872945785522461, -2.773503065109253], [2.9882142543792725, -2.9349143505096436], [0.5140730142593384, -0.450624018907547], [1.72993004322052, -1.6650402545928955], [3.9999496936798096, -3.95020341873

...

Epoch: 8, Loss:  0.6185241937637329
[[3.507873773574829, -3.473625421524048], [2.9006636142730713, -2.8975205421447754], [6.0362019538879395, -6.032756328582764], [1.4636598825454712, -1.5101072788238525], [2.484560489654541, -2.533799409866333], [2.578653573989868, -2.5737431049346924], [3.6049673557281494, -3.568963050842285], [3.3801746368408203, -3.409175157546997], [2.8940982818603516, -2.869946002960205], [2.2568392753601074, -2.28533673286438], [0.671455979347229, -0.7035664319992065], [1.4982908964157104, -1.5261626243591309], [2.285443067550659, -2.3203394412994385], [5.596468448638916, -5.565542697906494], [0.01447489857673645, -0.0910811573266983], [3.4644057750701904, -3.446115493774414], [4.790501117706299, -4.772666931152344], [5.012531280517578, -5.0167670249938965], [4.40703821182251, -4.380039215087891], [3.6273436546325684, -3.6106350421905518], [1.712401270866394, -1.7031701803207397], [1.4703757762908936, -1.4563313722610474], [-0.3655489385128021, 0.332001686096191

100%|██████████| 34/34 [00:36<00:00,  1.08s/it]


Epoch: 9, Loss:  0.38768261671066284
[[3.6752734184265137, -3.626359701156616], [2.4406845569610596, -2.3806521892547607], [5.70263147354126, -5.659484386444092], [0.20557492971420288, -0.17113259434700012], [0.9448182582855225, -0.906696081161499], [2.112730026245117, -2.059522867202759], [3.8438973426818848, -3.7958528995513916], [2.4158339500427246, -2.3748300075531006], [2.8115057945251465, -2.7617437839508057], [1.5189881324768066, -1.48319411277771], [-0.4724203944206238, 0.5187901258468628], [0.575404167175293, -0.5396642088890076], [1.2281919717788696, -1.1898863315582275], [5.660860538482666, -5.609445571899414], [-3.026155710220337, 3.088347911834717], [3.2536020278930664, -3.1954636573791504], [4.556898593902588, -4.501641750335693], [4.380565166473389, -4.332910537719727], [4.409729957580566, -4.365652084350586], [3.4454309940338135, -3.388481616973877], [1.3415462970733643, -1.279700756072998], [1.0589871406555176, -0.9884596467018127], [-1.3990669250488281, 1.447159171104

100%|██████████| 34/34 [00:36<00:00,  1.08s/it]


Epoch: 10, Loss:  0.14923733472824097
[[2.2652688026428223, -2.3026533126831055], [2.488619565963745, -2.4859628677368164], [5.716728210449219, -5.7227582931518555], [2.2928051948547363, -2.259963035583496], [3.6005349159240723, -3.5588488578796387], [1.838914155960083, -1.8400583267211914], [2.1854867935180664, -2.2267370223999023], [3.7602338790893555, -3.7359471321105957], [1.6203476190567017, -1.6472817659378052], [2.8059449195861816, -2.7883176803588867], [1.0682190656661987, -1.0360536575317383], [1.2549149990081787, -1.2395329475402832], [2.941704034805298, -2.9116716384887695], [4.266819000244141, -4.298003673553467], [1.812523365020752, -1.7178586721420288], [2.5987253189086914, -2.611361503601074], [3.9343080520629883, -3.945443630218506], [4.857948303222656, -4.848344326019287], [3.192798614501953, -3.224701404571533], [2.8902671337127686, -2.902785301208496], [1.0450750589370728, -1.044363260269165], [0.40151169896125793, -0.4056607484817505], [-0.006150506436824799, 0.0380

In [38]:
accuracy = metrics.accuracy_score(model_targets, model_labels)
f1_score_w_avg = metrics.f1_score(model_targets, model_labels, average='weighted')

print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Weighted) = {f1_score_w_avg}")

print(metrics.classification_report(model_targets, model_labels))

Accuracy Score = 0.8725925925925926
F1 Score (Weighted) = 0.8633092750870529
              precision    recall  f1-score   support

           0       0.90      0.95      0.93      1139
           1       0.63      0.45      0.52       211

    accuracy                           0.87      1350
   macro avg       0.77      0.70      0.72      1350
weighted avg       0.86      0.87      0.86      1350



# Tokenization testing

In [ ]:
print(tokenizer.decode([ 101, 28248, 35732, 22044,   118, 21631, 10345, 11101, 16602,   119,
        10159, 48502, 10107, 10108,   146, 16994, 10806,   117, 10231, 87150,
        22525, 22489, 11309, 10161, 21528, 10114, 63376, 11915, 10135,   108,
        10201, 35732, 22044,   123, 31081, 10169, 22528, 10114, 52824, 10123,
        11345,   119, 12882, 14796, 10944,   189, 11419, 51511, 51354, 10188,
        11049, 11309, 10161,   136,   102, 16938,   112,   188, 45476, 10114,
        23763, 10531,   119,   119,   119, 10678, 10114, 94992, 10219, 10111,
        23763, 10105, 11561, 40414,   119, 14453, 44096,   189, 11337, 10678,
        10114, 21852, 10479, 10301, 22489, 11426,   102]))

[CLS] Jalikattu - Ban or Allow. Lakhs of Idiots, uneducated Tamil ppl want to lift ban on # jalikattu 2play with animals to hurt them. So how can u expect gud from such ppl? [SEP] don't dare to say this... come to merina and say the same thing. Surely u will come to know who are Tamil people [SEP]


In [36]:
print(len(train_data[0]["token_type_ids"]))
print(len(train_data[0]["targets"]))
print(len(train_data[0]["ids"]))
print(len(train_data[0]["mask"]))

#print(train_data[0]["text_length"])
#print(train_data[0]["pt_length"])
#print(train_data[0]["topic_length"])

print(train_data[0]["token_type_ids"])
print(train_data[0]["ids"])


510
2
510
510
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 